In [6]:
import math

def torsiondesign(width, height, cover, db, tf, beff, phi_torsion, fcprime, fy, tu, vc, ds, smax_shear, sactual, av, s):
    """
    Perform torsion design checks. 
    Intermediate geometry and code checks.
    """

    depth = height - cover - ds - db/2.0 # mm

    # --- Compute intermediate geometry for torsional core ---
    x1  = width  - 2*(cover + db/2.0)   # mm
    y1  = height - 2*(cover + db/2.0)   # mm
    aoh = x1 * y1                       # mm^2
    ph  = 2*(x1 + y1)                   # mm

    # --- "Outer" cross-section area and perimeter ---
    acp = width*(height - tf) + beff*tf # mm^2
    pcp = 2*(beff + height)             # mm

    def torsionalthreshold():
        # φ T_th
        phitth = (phi_torsion
                  * 0.083
                  * math.sqrt(fcprime)
                  / 10.0
                  * (acp**2)/pcp
                  / 100.0)
        return phitth

    def crackingtorsionalmoment():
        # ϕTcr = 4*ϕTth
        phitth_val = torsionalthreshold()
        phitcr = 4.0 * phitth_val
        # min required transverse steel ratio
        min_at_s = max(0.031*math.sqrt(fcprime), 0.175)*width/fy
        return phitcr, min_at_s

    def torsionaleffect():
        phitth_val = torsionalthreshold()
        if tu < phitth_val:
            print("Torsional effects need not be considered.")
        else:
            print("Torsional effects MUST be considered.")

    def crushinglimit_combinedstress():
        # Example combined stress check
        crushinglimit = phi_torsion * (
            (vc / (width*depth)*10.0)
            + 0.66*math.sqrt(fcprime)
        )
        combinedstress = math.sqrt(
            (vc/(width*depth)*10.0)**2
            + tu*100.0*ph / (1.7*(aoh**2)*10.0)**2
        )
        if combinedstress < crushinglimit:
            print("Combined stress is OK (below crushing limit).")
        else:
            raise ValueError("Combined stress exceeds crushing limit (torsion).")

    def transversereinforcement():
        # A_t/s
        at_per_s = (
            tu*100.0 /
            (2.0 * phi_torsion * 0.85 * aoh * (fy/10.0))
        )
        # Compare vs. the code minimum
        _, min_at_s = crackingtorsionalmoment()
        totaltransverse = (av/s)/2.0 + at_per_s
        if totaltransverse < min_at_s:
            raise ValueError("Total transverse reinforcement < code minimum.")
        else:
            print("Transverse reinforcement is OK.")
        return at_per_s

    def stirrupsspacing():
        # smax = min(smax_shear, ph/8, 305)
        smax = min(smax_shear, ph/8.0, 305.0)
        if sactual > smax:
            raise ValueError("Stirrup spacing is larger than code maximum.")
        else:
            print("Stirrup spacing is OK.")

    def minimum_area_longitudinal():
        # get required torsional transverse ratio:
        at_per_s_val = transversereinforcement()
        # min_at = ...
        min_at = (
            0.42 * math.sqrt(fcprime)*acp/fy
            - max(at_per_s_val, 0.175*width/fy)*ph
        )
        req_at = at_per_s_val * ph
        if req_at < min_at:
            at_required = min_at
        else:
            at_required = req_at
        return at_required

    def barsize():
        at_required = minimum_area_longitudinal()
        areabar = math.pi*(db/2.0)**2
        if areabar > at_required/4.0:
            print("Chosen bar size is OK for longitudinal torsion steel.")
        else:
            raise ValueError("Bar area below the minimum needed.")

    torsionaleffect()
    stirrupsspacing()
    crushinglimit_combinedstress()
    barsize()
    print("\nTorsion design checks completed.\n")

if __name__ == "__main__":
    data = {
        "width":       600,  # total (outer) beam width, mm
        "height":      750,  # total (outer) beam height, mm
        "cover":       20,   # concrete cover, mm
        "db":          25,   # bar (reinforcement) diameter, mm
        "tf":          100,  # flange thickness or top thickness, mm
        "beff":        500,  # effective width of flange, mm
        "phi_torsion": 0.75, # strength reduction factor for torsion
        "fcprime":     28,   # concrete compressive strength, MPa
        "fy":          420,  # steel yield stress, MPa
        "tu":          40,   # factored torsion, kN-m
        "vc":          5,    # shear carried by concrete, kN
        "ds":          20,   # stirrups diameter, mm
        "smax_shear":  300,  # max allowable stirrup spacing (shear), mm
        "sactual":     200,  # actual stirrup spacing used, mm
        "av":          300,  # cross-sectional area of shear reinforcement * #legs
        "s":           150,  # stirrup spacing, mm
    }

    torsiondesign(
        data["width"],
        data["height"],
        data["cover"],
        data["db"],
        data["tf"],
        data["beff"],
        data["phi_torsion"],
        data["fcprime"],
        data["fy"],
        data["tu"],
        data["vc"],
        data["ds"],
        data["smax_shear"],
        data["sactual"],
        data["av"],
        data["s"]
    )

Torsional effects need not be considered.
Stirrup spacing is OK.
Combined stress is OK (below crushing limit).
Transverse reinforcement is OK.
Chosen bar size is OK for longitudinal torsion steel.

Torsion design checks completed.



In [ ]:
def torsiondesign ():
   
    x1 = width - 2 * (cover +db / 2)  # mm 
    y1 = height - 2 * (cover +db / 2) # mm
    aoh = x1 * y1 # mm2
    ph = 2 * (x1 + y1) # mm
    acp = width * (height - tf) + beff * tf # mm2
    pcp = 2 * (beff + height) # mm

    def torsionalthreshold ():
        phitth = phi_torsion * 0.083 * math.sqrt (fcprime) / 10 * math.pow (acp,2)/pcp / 100 # kN-m
        return phitth
    
    def crackingtorsionalmoment ():
        phitcr = 4 * phitth # kN-m
        min_atransverse_per_s = max (0.031 * math.sqrt (fcprime), 0.175) * width / fy # mm2/mm
        return phitcr, min_atransverse_per_s
    
    def torsionaleffect ():
        if tu > phitth:
            note:  'Torsional effects must be considered'
        else:
            raise ValueError ('Torsional effects must be considered')

    def crushinglimit_combinedstress ():
        crushinglimit = phi_torsion * (vc / (width * depth) * 10 + 0.66 * math.sqrt (fcprime)) # MPa
        combinedstress = math.sqrt (math.pow(vc/(width * depth) * 10,2)+(tu*100*ph/math.pow(1.7*math.pow(aoh,2)*10,2)) # MPa
        if combinedstress < crushinglimit:
            note: 'Okay'
        else:
            raise ValueError ('Torsional effects must be considered')

    def transversereinforcement ():
        at_per_s = tu * 100 / (2 * phi_torsion * 0.85 * aoh * fy / 10) # mm2/mm
        totaltransverse = av/s /2 + at/s # mm2/mm
        if totaltransverse < min_atransverse_per_s:
            raise ValueError ('Total transverse is less than minimum')
        else:
            note: 'Okay'
    
    def stirrupsspacing ():
        smax = min (smax_shear, ph/8, 305) # mm ACI 22.7.6.1
        if smax < sactual:
            raise ValueError ('Stirrups spacing is more than maximum')
        else:
            note: 'Okay'
    
    def minimum_area_longitudinal ():
        min_at = 0.42 * math.sqrt (fcprime) * acp / fy - max (at_per_s, 0.175 * width / fy) * ph # mm2
        req_at = at_per_s * ph # mm2
        if req_at < min_at:
            at = min_at
        else:
            at = req_at

    def barsize ():
        areabar = math.pi * math.pow (bar_size/2,2) # mm2
        if areabar > at/4:
            note: "okay"
        else:  
            raise ValueError ('Area bar is less than limit')